In [ ]:
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from src.Evaluation.Evaluator import EvaluatorHoldout
import scipy.sparse as sps
import random
import os
import numpy as np
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)

URM_train = sps.load_npz('Dataset/URM_train.npz')
URM_test_complete = sps.load_npz('Dataset/URM_test.npz')

URM_all=URM_train + URM_test_complete

In [ ]:


evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
def objective(trial):
    num_factors = trial.suggest_int('num_factors', 5, 300)
    l1_ratio = trial.suggest_float('l1_ratio',1e-7, 1.0, log=True)
    solver_beta_loss = trial.suggest_categorical('solver_beta_loss', ["coordinate_descent:frobenius", "multiplicative_update:frobenius"])
    init_type = trial.suggest_categorical('init_type', ["random", "nndsvda"])

    nmf_recommender = NMFRecommender(URM_train)
    nmf_recommender.fit(
        num_factors=num_factors,
        l1_ratio=l1_ratio,
        solver_beta_loss=solver_beta_loss,
        init_type=init_type,
        verbose=False
    )

    result_df, _ = evaluator.evaluateRecommender(nmf_recommender)
    return result_df.loc[10, "MAP"]

In [ ]:
import optuna
from optuna.samplers import RandomSampler

database_path='hypertuning_results/tuning_NMF.db'

study=optuna.load_study(
    study_name='hyperparameters_NMF',
    storage=f'sqlite:///{database_path}'

)
study.sampler=RandomSampler(seed=SEED)
study.optimize(objective, n_trials=150, n_jobs=4)